# Beispiel 2: Library of Congress API

Die Library of Congress (LOC) bietet eine Reihe sehr gut dokumentierter APIs zur Abfrage von Metadaten, Dateien und Volltexten aus dem Bestand der Bibliothek. Eine davon ist die API der Sammlung US-amerikanischer historischer Zeitungen Chronicling America. Diese API werden wir in dieser Stunde kennenlernen.

- Übersicht über die LOC APIs: https://guides.loc.gov/digital-scholarship/accessing-digital-materials#s-lib-ctab-26648178-2
- Dokumentation zur Chronicling America API: https://chroniclingamerica.loc.gov/about/api/

Zunächst machen wir uns mit der Chronicling America API vertraut. Welche Daten können darüber abgefragt werden?

Für unser Beispiel werden wir die Volltexte zu allen Ergebnissen einer Suche nach Schlüsselwörtern in den Volltexten der Zeitungen abfragen und herunterladen (Abschnitt "Searching the directory and newspaper pages using OpenSearch"). Die Volltexte sind mithilfe von OCR-Verfahren erstellt, also mithilfe von automatischer Bilderkennung. Unsere Suchabfrage liefert also nur diejenigen Zeitungen, in denen die Suchwörter korrekt erkannt wurden.

### Vorbereitung

In [ ]:
# wir müssen zunächst die Anaconda Einstellungen ändern, damit wir das Paket ratelimit installieren könenn:
# https://stackoverflow.com/questions/48493505/packagesnotfounderror-the-following-packages-are-not-available-from-current-cha
# import sys
# !conda config --append channels conda-forge

In [ ]:
# Paket ratelimit installieren
# import sys
# !conda install --yes --prefix {sys.prefix} ratelimit

In [ ]:
# Pakete importieren
import requests
import os
import math
import time
# from ratelimit import limits, RateLimitException, sleep_and_retry

### Exploration der Chronicling America API

Wie in der letzten Stunde müssen wir zur Abfrage von Daten wieder eine URI nach den Vorgaben der API Dokumentation zusammensetzen.

Suchabfragen können mit einem `?` an die URL https://chroniclingamerica.loc.gov/search/pages/results/ angefügt werden.
Es gibt laut [Dokumentationsseite](https://chroniclingamerica.loc.gov/about/api/) drei verschiedene Abfrageparameter:

- andtext: the search query
- format: 'html' (default), or 'json', or 'atom' (optional)
- page: for paging results (optional)

Diese Parameter werden wir uns der Reihe nach ansehen.

Parameter andtext

In [ ]:
# Der andtext Parameter: Volltexte nach Schlagwörtern oder Phrasen durchsuchen
# Suche nach Schlagwörtern book AND review
url_1 = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=book+review"
url_2 = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=book%20review"

Eine aufmerksame Betrachtung der Ergebnisse der Suche nach einem Suchbegriff über die Suchmaske der Website https://chroniclingamerica.loc.gov zeigt, dass die Suche über die Suchmaske genau dieselben Ergebnisse liefert wie die API-Abfrage. Das ist nicht erstaunlich, denn die URL, die beim Verwenden der Suchmaske generiert wird, ist ganz ähnlich aufgebaut wie die URL, die wir für die API-Abfrage erstellen, mit dem einzigen Unterschied, dass die Suchparameter etwas anders aussehen:

:::{figure-md}
<img src="loc_ca_suche.png" alt="Suchmaske Chronicling America" class="bg-transparent" width="80%">

Einfache Suche über die Suchmaske der Seite Chronicling America.
:::

Wir können diese URL nutzen, um den andtext-Parameter besser zu verstehen. Wenn wir in der einfachen Suche nach dem Suchbegriff "book review" suchen, dann steht in der URL "book+review". Wenn wir stattdessen die Erweiterte Suche (Tab Advanced Search) verwenden und nach einer Phrase suchen, dann steht in der URL der Zusatz "&phrasetext=book+review":

:::{figure-md}
<img src="loc_ca_erweiterte_suche.png" alt="Erweiterte Suche Chronicling America" class="bg-transparent" width="80%">

Erweiterte Suche über die Suchmaske der Seite Chronicling America.
:::

Tatsächlich akzeptiert auch die Chronicling America API eine Abfrage-URI mit dem Zusatz &phrasetext:

In [ ]:
# Suche nach Phrase "book review"
url_3 = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=&phrasetext=book+review"
# woher weiß ich das: manuelle Suche über "advanced search" und URL untersuchen
# https://chroniclingamerica.loc.gov/search/pages/results/?dateFilterType=yearRange&date1=1770&date2=1963&language=&ortext=&andtext=&phrasetext=book+review&proxtext=&proxdistance=5&rows=20&searchType=advanced
search_results = requests.get(url_3)
# search_results

Parameter format

In [ ]:
# Der format-Parameter: Ergebnisse im JSON-Format abfragen
# https://chroniclingamerica.loc.gov/search/pages/results/?andtext=&phrasetext=book+review&format=json
# erste Seite der Suchergebnisse: 20 Ergebnisse je Seite
url = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=&phrasetext=book+review&format=json"
search_results = requests.get(url)
# search_results

```{note}
JSON im Chrome Browser ansehen

Zur Ansicht der JSON-Datei im Chrome Browser können wir wieder auf die Entwicklertools zurückgreifen. Die Standardansicht ist nämlich sehr schwer lesbar, weil der JSON-String nicht formatiert ist. Um eine formatierte Ansicht zu erhalten, befolgt die folgenden Schritte: Entwicklertools öffnen -> "Sources"-Tab auswählen-> Link anklicken
```

Parameter page

In [ ]:
# Der page Parameter: Nur die ausgewählte Ergebnisseite abfragen
first_page = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=&phrasetext=book+review&format=json&page=1"
first_page_results = requests.get(first_page)
# first_page_results

Per Default werden immer die ersten 20 Suchergebnisse (also die erste Seite der Suchergebnisse) ausgegeben, wenn der page-Parameter weggelassen wird:

In [ ]:
default_page = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=&phrasetext=book+review&format=json"
default_page_results = requests.get(default_page)
first_page_results.content == default_page_results.content

Mit diesem Wissen könenn wir eine Testabfrage durchführen.

Für unsere Abfragen wählen wir JSON als Rückgabeformat aus, weil wir den JSON-String bequem parsen können, indem wir den String in ein Python Dictionary umwandeln:

In [ ]:
url = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=&phrasetext=book+review&format=json"
# JSON-String in Python Dictionary umwandeln
search_results = requests.get(url).json()
print(len(search_results["items"]))

Das Dictionary enthält einen Schlüssel "items" mit einer Liste der Suchergebnisse als Wert. Die Suchergebnisse sind selbst als Dictionaries organisiert. Jedes Suchergebnis-Dictionary enthält einen Schlüssel "ocr_eng" mit den Volltexten:

In [ ]:
# erstes Suchergebnis auf der ersten Seite der Suchergebnisse
search_results["items"][0]["ocr_eng"]

Um die Volltexte für alle Suchergebnisse auf der ersten Seite abzurufen und zu speichern, können wir eine for-Schleife entwerfen:

In [ ]:
# Volltexte für die gesamte erste Seite der Suchergebnisse speichern
# items = search_results["items"]

# for item in items:
#     ocr_text = item["ocr_eng"]
#     title = item["title"]
#     date = item["date"]
#     with open(f"{title}_{date}.txt", "w") as file:
#         file.write(ocr_text)

Das wollen wir jetzt für alle Suchergebnisse auf allen Seiten reproduzieren.

### Abfrage aller Volltexte mit "book review"

Zunächst legen wir in unserem aktuellen Arbeitsverzeichnis (=Ordner, in dem die Jupyter Notebooks liegen) ein neues Verzeichnis an, in dem wir die Volltexte abspeichern werden:

In [ ]:
# neues Verzeichnis anlegen: in diesem Ordner werden die Textdateien gespeichert
# if not os.path.exists("loc_ocr"):
#     os.makedirs("loc_ocr")
# Arbeitsverzeichnis wechseln: Die Texte sollen im Verzeichnis "spokentext_corpus" gespeichert werden
# os.chdir(os.path.join(os.getcwd(), "loc_ocr"))

Wie gehen wir vor, um jetzt unsere for-Schleife oben nacheinander auf alle Ergebnisseiten anzuwenden?
Eine Idee wäre die Verwendung einer while Schleife mit HTTP Antwort != 200 als break-Bedingung. Diese Strategie ist aber nur anwendbar, wenn beim Abruf einer ungültigen Seite eine HTTP-Antwort ungleich 200 zurückgegeben wird. Das müssen wir zunächst überprüfen: Was passiert, wenn eine nicht existierende Seite aufgerufen wird?
Als Beispiel rufen wir die Seite https://chroniclingamerica.loc.gov/search/pages/results/?rows=20&format=json&sequence=0&phrasetext=book+review&andtext=&page=1640 auf.

Tatsächlich gibt es eine Umleitung auf Seite 1 mit einem gültigen HTTP-Statuscode. Wir können also in diesem Fall die Strategie mit der while-Schleife nicht verwenden.

Eine andere Idee ist die Verwendung einer for-Schleife. Dazu müssen wir aber die Gesamtzahl der Ergebnisseiten kennen. Die Gesamtzahl der Ergebnisseiten können wir aber einfach ermitteln:

In [ ]:
# Gesamtanzahl der Ergebnisseiten ermitteln: Anzahl der Ergebnisse durch Anzahl der Ergebnisse pro Seite teilen
url = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=&phrasetext=book+review&format=json"
search_results = requests.get(url).json()
pages_float = search_results["totalItems"] / search_results["itemsPerPage"]
pages = math.ceil(pages_float)+1 # aufrunden und 1 addieren
pages

Zu der Gesamtzahl der Seiten addieren wir 1, da wir später die range(1, n)-Funktion verwenden wollen, welche eine Integersequenz von Zahl 1 bis Zahl n-1 generiert.

Unsere for-Schleife sieht dann so aus:

In [ ]:
# Volltexte zu allen Ergebnissen von allen Ergebnisseiten speichern
# url = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=&phrasetext=book+review&format=json&page="

# for page in range(1, pages):
#     request_url = url + str(page)
#     response = requests.get(request_url).json()
#     # for-Schleife für eine einzelne Ergebnisseite einsetzen
#     items = response["items"]
#     for item in items:
#         ocr_text = item["ocr_eng"]
#         title = item["title"]
#         date = item["date"]
#         with open(f"{title}_{date}.txt", "w") as file:
#             file.write(ocr_text)

Aber Achtung! Beim Ausführen des Codes oben gibt es nach einigen Schleifendurchläufen eine Fehlermeldung: JSONDecodeError: Expecting value: line 1 column 1 (char 0). Die Fehlermeldung entsteht dann, wenn die HTTP-Anfrage keine erfolgreiche Antwort liefert. Das liegt mit großer Wahrscheinlichkeit daran, dass wir uns nicht an die Einschränkungen der LOC gehalten haben und die HTTP-Anfrage dadurch ab einem bestimmten Punkt abgelehnt wird. Wenn wir dann versuchen, den Antwortbody mithilfe der .json()-Methode in ein Python Dictionary umzuwandeln, teilt der Python interpreter uns mit, dass das nicht möglich ist, weil wir die Methode nicht auf einen gültigen JSON-String angewendet haben.

Bei der Abfrage von sehr vielen Seiten müssen wir uns also nach den Einschränkungen der LOC richten. Die LOC hat Einschränkungen für die der Chronicling America API übergeordnete Seite loc.gov festgelegt, und wir können vermuten, dass die Einschränkungen auch für die Chronicling America API gelten:  https://www.loc.gov/apis/json-and-yaml/working-within-limits

Um auf der sicheren Seite zu sein, richten wir uns nach der restriktivsten Vorgabe, nach der nur 20 Abfragen alle 10 Sekunden erlaubt sind.

Wie können wir also die HTTP-Abfragen auf 20 Abfragen je 10 Sekunden einschränken? Was wir brauchen nennt sich "rate limiting", also eine Begrenzung der Abfragerate, die wir in unseren Code einbauen müssen.

Um die Abfragerate einzuschränken, gibt es zwei etablierte Möglichkeiten:

1) **Funktion `time.sleep()` aus dem Paket time**. Die Funktion time.sleep(x) kann in den Schleifenkörper einer for-Schleife eingefügt werden, um den nächsten Schleifendurchlauf um x Sekunden zu verzögern. Diese Methode ist einstiegsfreundlich, aber ungenau, weil die Laufzeit der Schleife selbst nicht in die Wartezeit mit einbezogen wird, sodass der nächste Schleifendurchlauf länger als notwendig verzögert wird.
2) **Python Dekoratoren aus dem Paket ratelimit**. Wesentlich effizienter und eleganter ist die Verwendung von sogenannten Python Dekoratoren bzw. Decorators. Das Paket ratelimit bietet zwei solche Dekoratoren, die dazu verwendet werden können, um zu registrieren, wie häufig eine Funktion nacheinander aufgerufen wird, und die ab einer bestimmten Anzahl wiederholter Aufrufe eine Wartepause erzwingen. Um Decorators verwenden zu können, müssen wir unsere Abfrage jedoch in eine Funktion verpacken.

```{note}
Python Dekoratoren (Decorators)

> A decorator in Python is a function that accepts another function as an argument. The decorator will usually modify or enhance the function it accepted and return the modified function. This means that when you call a decorated function, you will get a function that may be a little different that may have additional features compared with the base definition.

Quelle: [Michael Droscill (2017).](https://python101.pythonlibrary.org/chapter25_decorators.html)

Dekoratoren beruhen auf einem komplexen Konzept und wir können hier nicht tiefer einsteigen, aber wenn die ein oder andere Person doch etwas tiefer einsteigen will, kann ich diese beiden Ressourcen empfehlen:
- Primer on Python Decorators, https://realpython.com/primer-on-python-decorators/
- Python Decorators in 15 Minutes, https://www.youtube.com/watch?v=r7Dtus7N4pI

Bei der Verwendung der Dekoratoren aus dem Paket ratelimit verwenden wir diese Anleitung von Akshay Ranganath:
- Rate Limiting with Python, https://akshayranganath.github.io/Rate-Limiting-With-Python/
```

In [ ]:
# 1) Rate limiting mit time.sleep()

# url = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=&phrasetext=book+review&format=json"
# search_results = requests.get(url).json()
# pages_float = search_results["totalItems"] / search_results["itemsPerPage"]
# pages = math.ceil(pages_float) # aufrunden

# url = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=&phrasetext=book+review&format=json&page="
# for page in range(1, pages):
#     request_url = url + str(page)
#     response = requests.get(request_url).json()
#     # for-Schleife für eine einzelne Ergebnisseite einsetzen
#     items = response["items"]
#     for item in items:
#        ocr_text = item["ocr_eng"]
#         title = item["title"]
#         date = item["date"]
#         with open(f"{title}_{date}.txt", "w", encoding="utf-8") as file:
#             file.write(ocr_text)
#     time.sleep(10)

In [ ]:
# 2) Rate limiting mit Python decorators

# url = "https://chroniclingamerica.loc.gov/search/pages/results/?andtext=&phrasetext=book+review&format=json"
# search_results = requests.get(url).json()
# pages_float = search_results["totalItems"] / search_results["itemsPerPage"]
# pages = math.ceil(pages_float) # aufrunden
# url = url + "&page="

# TEN_SECONDS = 10
# CALLS_PER_TEN_SECONDS = 20 # 20 Abfragen in 10 Sekunden

# @sleep_and_retry
# @limits(calls=CALLS_PER_TEN_SECONDS, period=TEN_SECONDS)
# def get_fulltext(url):
#     response = requests.get(url).json()
#     # for-Schleife für eine einzelne Ergebnisseite einsetzen
#     items = response["items"]
#     for item in items:
#         ocr_text = item["ocr_eng"]
#         title = item["title"]
#         date = item["date"]
#         with open(f"{title}_{date}.txt", "w", encoding="utf-8") as file:
#             file.write(ocr_text)

# for page in range(1, pages):
#     request_url = url + str(page)
#     get_fulltext(request_url)


```{note}
Konstanten (Constants)

Im Code oben verwenden wir Großbuchstaben, um die beiden Variablen `CALLS_PER_TEN_SECONDS` und `TEN_SECONDS` zu benennen. Diese Schreibweise hat sich in Python für Konstanten etabliert, also für Variablen, deren Wert sich im Programmverlauf nicht ändert.
```

### Quellen

```{bibliography}
   :list: enumerated
   :filter: keywords % "decorators" or keywords % "loc"
```